In [114]:
import numpy as np
import pandas as pd
from numpy import nan
import pickle
pd.set_option('display.max_rows', 200)

## Data Cleaning

### 1. load data

In [115]:
comprehensive = pd.read_stata('../Data/data/Comprehensive-Sample.dta')
comprehensive

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,aspo,wildcat,endowment,pop_maddison,...,logmountain,logoutreg,dincidence2COW,dmilexpSIPRI,dincidenceU,dcoup,decade,wildcatsample,lowincsample,opec
0,1929.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.041836,0.127953,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
1,1930.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.041836,0.032496,-1.0,NaN,NaN,NaN,1.0,NaN,1.0,0.0
2,1931.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.041836,0.032496,0.0,NaN,NaN,NaN,1.0,NaN,1.0,0.0
3,1932.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.041836,0.032496,0.0,NaN,NaN,NaN,1.0,NaN,1.0,0.0
4,1933.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.041836,0.127844,0.0,NaN,NaN,NaN,1.0,NaN,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,2004.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,10962.026367,...,-0.016094,0.177880,0.0,NaN,0.0,0.0,8.0,NaN,1.0,0.0
17916,2005.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,11115.380859,...,-0.016094,0.187731,NaN,NaN,0.0,0.0,8.0,NaN,1.0,0.0
17917,2006.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,11288.252930,...,-0.016094,0.167053,NaN,NaN,0.0,0.0,8.0,NaN,1.0,NaN
17918,2007.0,894,0.0,NaN,0.0,NaN,NaN,NaN,0.0,11477.447266,...,-0.016094,0.172984,NaN,NaN,0.0,0.0,8.0,NaN,1.0,NaN


In [116]:
comprehensive.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17920 entries, 0 to 17919
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        17920 non-null  float32
 1   numcode                     17920 non-null  int16  
 2   oilreserves_full            14307 non-null  float32
 3   oilreserves                 12519 non-null  float32
 4   oilreserves_public          9950 non-null   float32
 5   newdiscovery_aspo           12354 non-null  float32
 6   aspo                        4736 non-null   float32
 7   wildcat                     12354 non-null  float32
 8   endowment                   14160 non-null  float32
 9   pop_maddison                10441 non-null  float64
 10  ecgrowth                    9341 non-null   float32
 11  efrac                       390 non-null    float32
 12  lfrac                       390 non-null    float32
 13  rfrac                       390

In [117]:
aspo = pd.read_stata('../Data/data/ASPO-Sample.dta')

In [118]:
'discoveryaspoPC' in aspo.columns

True

In [119]:
aspo[['wildcat','year']][~aspo['wildcat'].isna()].query('wildcat > 0')

,wildcat,year
1,1.0,1930.0
28,1.0,1957.0
31,1.0,1960.0
34,3.0,1963.0
35,1.0,1964.0
...,...,...
5097,10.0,1986.0
5098,3.0,1987.0
5099,4.0,1988.0
5100,16.0,1989.0


### 2. function to do transformation

In [120]:
# Create a sample dataframe
df = pd.DataFrame({"country": np.random.choice(["A", "B", "C"], size=20),
                   "year": np.arange(20),
                   "lognum1": np.random.randint(1, 4, size=20),
                   "num2": np.random.randint(20000, 90000, size=20),
                   "num3": np.random.randint(20000, 90000, size=20),
                   "num4": np.random.randint(20000, 90000, size=20)})

In [121]:
df = df.sort_values(by=['country','year'], ascending=True)

In [122]:
df

,country,year,lognum1,num2,num3,num4
1,A,1,3,71258,47220,30010
3,A,3,2,32582,45026,59495
2,B,2,2,70568,40574,47122
5,B,5,1,33295,29150,72708
6,B,6,3,46348,21690,63714
12,B,12,3,20949,72145,56131
17,B,17,2,79260,25096,84023
18,B,18,1,77722,72143,87114
19,B,19,2,27059,73620,59303
0,C,0,2,79206,70653,56379


In [123]:
def transform_df(df, groupby_cols, log_cols, diff_cols, pct_change_cols, binarize_cols):
    
    new_df = df.copy()
    new_cols = []

    for i, col_name in enumerate(log_cols):
        new_col_name = col_name[3:] # to get rid of 'log' in the column name
        new_df[new_col_name] = np.exp(new_df[col_name])
        log_cols[i] = new_col_name

    all_cols = log_cols + pct_change_cols
    
    for col_name in all_cols:
        
        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df.groupby(groupby_cols)[col_name].pct_change()
        new_cols.append(diff_name)

        if col_name in binarize_cols:
            binarize_name = col_name + '_binarize'
            new_df[binarize_name] = np.NAN
            new_df.loc[new_df[diff_name] > 0.1, binarize_name] = 1
            new_df.loc[new_df[diff_name] <= 0.1, binarize_name] = 0
            new_df.loc[pd.isnull(new_df[diff_name]), binarize_name] = np.nan
            new_cols.append(binarize_name)

    for col_name in diff_cols:

        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df[col_name] - new_df.groupby(groupby_cols)[col_name].shift(1)
        new_cols.append(diff_name)
            
    return new_df[new_cols]

In [124]:
transform_df(df, ['country'], ['lognum1'], ['num4'], ['num2','num3'], ['num3'])

,num1_diff,num2_diff,num3_diff,num3_binarize,num4_diff
1,NaN,NaN,NaN,NaN,NaN
3,-0.632121,-0.542760,-0.046463,0.0,29485.0
2,NaN,NaN,NaN,NaN,NaN
5,-0.632121,-0.528186,-0.281560,0.0,25586.0
6,6.389056,0.392041,-0.255918,0.0,-8994.0
12,0.000000,-0.548006,2.326187,1.0,-7583.0
17,-0.632121,2.783474,-0.652145,0.0,27892.0
18,-0.632121,-0.019404,1.874681,1.0,3091.0
19,1.718282,-0.651849,0.020473,0.0,-27811.0
0,NaN,NaN,NaN,NaN,NaN


--

### 3. comprehensive - cleaning

In [125]:
v_lst = ['onset2COWCS',
         'onsetUCS',
         'coup',
         'periregular',
         'milexp_pergdpSIPRI',
         #'logmilexgdpSIPRI',
         'numcode',
         'year',
         
         #'logGDP_M',
         'ecgrowth',
         'pop_maddison',
         #'logpop_M',
         'logpopdens', ####
         'democracy',
         'logmountain',
         'ethnic_fractionalization',
         'religion_fractionalization',
         'language_fractionalization',
         'leg_british',
         'no_transition',
         'wildcat',
         
         'oilreserves', # added by Wanran on Nov 28
         'valoilres', # value of oil reserves
         #'logvaloilres',
         'valoilres_public',
         #'logvaloilres_public',
         'oilpop',
         #'logoilres',
         'valoilres_impute',
         #'logvaloilres_impute',
         'oilpop_impute',
         #'logoilres_impute',
         'out_regdisaster']
         #'logoutreg'

In [126]:
# variables that need transformation
# transform_lst = ['logmilexgdpSIPRI', # defense burden
#                 'logpop_M',
#                 'logpopdens',
#                 'democracy', # democracy index
#                 'wildcat', # wildcat drilling 
#                 'logoutreg', # out of region disaster 
#                 'logvaloilres',
#                 'logvaloilres_public',
#                 'logoilres',
#                 'logvaloilres_impute',
#                 'logoilres_impute'
#                 ]

transform_lst = ['milexp_pergdpSIPRI',
                'pop_maddison',
                'logpopdens',
                'democracy',
                'wildcat',
                'out_regdisaster',
                #'oilreserves',
                'valoilres', # value of oil reserves
                'valoilres_public',
                'oilpop',
                'valoilres_impute',
                'oilpop_impute']

In [127]:
# variables that need to take exp and then compute pct change
log_lst = ['logpopdens']

# variables that need to take diff directly
diff_lst = ['milexp_pergdpSIPRI']

# variables that need to take pct change directly
pct_change_lst = ['pop_maddison', 
                'democracy',
                'wildcat',
                'out_regdisaster',
                #'oilreserves',
                'valoilres', # value of oil reserves
                'valoilres_public', # value of oil reserves from public data
                'oilpop', # oil reserves per capita in million barrels per 1000 persons
                'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
                'oilpop_impute'] # oil reserves per capita - imputed

# a subset of pct_change_lst
# variables that need to be binarized
binarize_lst = ['valoilres', # value of oil reserves
              'valoilres_public', # value of oil reserves from public data
              'oilpop', # oil reserves per capita in million barrels per 1000 persons
              'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
              'oilpop_impute'] # oil reserves per capita - imputed

# variables that don't need transformation
untransformed_lst = ['onset2COWCS',
                    'onsetUCS',
                    'coup',
                    'periregular',
                    'numcode',
                    'year',
                    'ecgrowth',
                    'logmountain',
                    'ethnic_fractionalization',
                    'religion_fractionalization',
                    'language_fractionalization',
                    'leg_british',
                    'no_transition']

In [128]:
comprehensive = comprehensive.replace(0, 0.1**5)

In [129]:
comprehensive['onset2COWCS'] = comprehensive.onset2COWCS.replace(0.1**5, 0)
comprehensive['onsetUCS'] = comprehensive.onsetUCS.replace(0.1**5, 0)
comprehensive['leg_british'] = comprehensive.leg_british.replace(0.1**5, 0)

In [130]:
comprehensive = comprehensive.sort_values(by=['numcode','year'], ascending=True)

In [131]:
new_df = transform_df(df = comprehensive,
                      groupby_cols = ['numcode'], 
                      log_cols = log_lst, 
                      diff_cols = diff_lst, 
                      pct_change_cols = pct_change_lst, 
                      binarize_cols = binarize_lst)
                      

In [132]:
comprehensive_new = pd.concat([comprehensive[untransformed_lst], new_df], axis = 1)

In [133]:
comprehensive_new.shape

(17920, 29)

In [134]:
comprehensive_new.columns

Index(['onset2COWCS', 'onsetUCS', 'coup', 'periregular', 'numcode', 'year',
       'ecgrowth', 'logmountain', 'ethnic_fractionalization',
       'religion_fractionalization', 'language_fractionalization',
       'leg_british', 'no_transition', 'popdens_diff', 'pop_maddison_diff',
       'democracy_diff', 'wildcat_diff', 'out_regdisaster_diff',
       'valoilres_diff', 'valoilres_binarize', 'valoilres_public_diff',
       'valoilres_public_binarize', 'oilpop_diff', 'oilpop_binarize',
       'valoilres_impute_diff', 'valoilres_impute_binarize',
       'oilpop_impute_diff', 'oilpop_impute_binarize',
       'milexp_pergdpSIPRI_diff'],
      dtype='object')

In [135]:
# potential selection of variables
len((comprehensive_new[['onset2COWCS',
                   'valoilres_binarize',
                   'ecgrowth',
                   'pop_maddison_diff',
                   'popdens_diff',
                   'democracy_diff',
                   'logmountain',
                   'ethnic_fractionalization',
                   'religion_fractionalization',
                   'language_fractionalization',
                   'leg_british',
                   'numcode',
                   'year']].dropna())['numcode'].unique())

103

In [136]:
#comprehensive_new[['oilpop_impute_binarize','year']].groupby('oilpop_impute_binarize').count()

In [137]:
comprehensive_new.to_pickle("../Data/data/comprehensive_new.pkl")  

In [138]:
comprehensive_new['valoilres_binarize'].describe()

count    7873.000000
mean        0.111266
std         0.314482
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: valoilres_binarize, dtype: float64

In [139]:
comprehensive['valoilres_impute'].describe()

count     7620.000000
mean       124.457756
std        849.143555
min          0.000010
25%          0.000010
50%          0.000010
75%          3.599900
max      21176.099609
Name: valoilres_impute, dtype: float64

In [140]:
(comprehensive_new['valoilres_impute_binarize'] == comprehensive_new['valoilres_binarize']).sum()

7650

In [141]:
for col_name in comprehensive_new.columns:
    print(comprehensive_new[col_name].describe())

count    14912.000000
mean         0.008047
std          0.089336
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: onset2COWCS, dtype: float64
count    11848.000000
mean         0.019750
std          0.139156
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: onsetUCS, dtype: float64
count    12855.000000
mean         0.055708
std          0.275568
min          0.000010
25%          0.000010
50%          0.000010
75%          0.000010
max          5.000000
Name: coup, dtype: float64
count    5758.000000
mean        0.207033
std         0.315318
min         0.000010
25%         0.000010
50%         0.000010
75%         0.375000
max         1.000000
Name: periregular, dtype: float64
count    17920.000000
mean       431.250000
std        252.825241
min          4.000000
25%        213.500000
50%        429.000000
75%        642.250000
max        894.000000
Na

In [142]:
comprehensive_new = pd.read_pickle('../Data/data/comprehensive_new.pkl')

In [143]:
len(comprehensive_new['numcode'].unique())

224

In [144]:
comprehensive_new_full = pd.concat([comprehensive, new_df], axis = 1)

In [145]:
comprehensive_new_full.to_pickle("../Data/data/comprehensive_new_full.pkl")  

### 4. ASPO - cleaning

In [146]:
aspo.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5120 entries, 0 to 5119
Data columns (total 108 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    year                        float32
 1    numcode                     int16  
 2    oilreserves_full            float32
 3    oilreserves                 float32
 4    oilreserves_public          float32
 5    newdiscovery_aspo           float32
 6    imputedzero                 float32
 7    aspo                        float32
 8    wildcat                     float32
 9    endowment                   float32
 10   ecgrowth                    float32
 11   country                     object 
 12   incidence2COW               float64
 13   onset2COWCS                 float32
 14   incidenceU                  float32
 15   onsetUCS                    float32
 16   Fearon_war                  float64
 17   onset_FearonCS              float32
 18   Sambanis_war                float64
 19   onse

In [147]:
def transform_df(df, groupby_cols, log_cols, log_diff_cols, pct_change_cols, binarize_cols):
    
    new_df = df.copy()
    new_cols = []

    for i, col_name in enumerate(log_cols):
        new_col_name = col_name[3:] # to get rid of 'log' in the column name
        new_df[new_col_name] = np.exp(new_df[col_name])
        log_cols[i] = new_col_name

    for col_name in log_diff_cols:
        
        new_col_name = col_name[3:]
        new_df[new_col_name] = np.exp(new_df[col_name])
        diff_name = new_col_name + '_diff'
        new_df[diff_name] = new_df[new_col_name] - new_df.groupby(groupby_cols)[new_col_name].shift(1)
        new_cols.append(diff_name)

    all_cols = log_cols + pct_change_cols
    
    for col_name in all_cols:
        
        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df.groupby(groupby_cols)[col_name].pct_change()
        new_cols.append(diff_name)

        if col_name in binarize_cols:
            binarize_name = col_name + '_binarize'
            new_df[binarize_name] = np.NAN
            new_df.loc[new_df[diff_name] > 0.1, binarize_name] = 1
            new_df.loc[new_df[diff_name] <= 0.1, binarize_name] = 0
            new_df.loc[pd.isnull(new_df[diff_name]), binarize_name] = np.nan
            new_cols.append(binarize_name)
            
    return new_df[new_cols]

In [148]:
# variables that need to take exp and then compute pct change
log_lst = ['logpopdens',
          'logpop_M',
          'logvaloilres',
          'logvaloilres_impute'
          ]

# variables that need to take diff directly
log_diff_lst = ['logmilexgdpSIPRI']

# variables that need to take pct change directly
pct_change_lst = ['democracy',
                'wildcat',
                'out_regdisaster',
                'oilreserves',
                
                
                
                'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
                'oilpop_impute'] # oil reserves per capita - imputed

# a subset of pct_change_lst
# variables that need to be binarized
binarize_lst = ['valoilres', # value of oil reserves
              
              
              'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
              'oilpop_impute'] # oil reserves per capita - imputed

# variables that don't need transformation
untransformed_lst = ['onset2COWCS',
                    'onsetUCS',
                    'coup',
                    'periregular',
                    'numcode',
                    'year',
                    'ecgrowth',
                    'logmountain',
                    'ethnic_fractionalization',
                    'religion_fractionalization',
                    'language_fractionalization',
                    'leg_british',
                    'no_transition']

In [149]:
aspo = aspo.replace(0, 0.1**5)

In [150]:
aspo['onset2COWCS'] = aspo.onset2COWCS.replace(0.1**5, 0)
aspo['onsetUCS'] = aspo.onsetUCS.replace(0.1**5, 0)
aspo['leg_british'] = aspo.leg_british.replace(0.1**5, 0)

In [151]:
aspo = aspo.sort_values(by=['numcode','year'], ascending=True)

In [152]:
new_df_aspo = transform_df(df = aspo,
                        groupby_cols = ['numcode'], 
                        log_cols = log_lst, 
                        log_diff_cols = log_diff_lst, 
                        pct_change_cols = pct_change_lst, 
                        binarize_cols = binarize_lst)
                      

In [153]:
new_df_aspo.columns

Index(['milexgdpSIPRI_diff', 'popdens_diff', 'pop_M_diff', 'valoilres_diff',
       'valoilres_binarize', 'valoilres_impute_diff',
       'valoilres_impute_binarize', 'democracy_diff', 'wildcat_diff',
       'out_regdisaster_diff', 'oilreserves_diff', 'valoilres_impute_diff',
       'valoilres_impute_binarize', 'oilpop_impute_diff',
       'oilpop_impute_binarize'],
      dtype='object')

In [154]:
aspo_new = pd.concat([aspo[untransformed_lst], new_df_aspo], axis = 1)

In [155]:
aspo_new.to_pickle("../Data/data/aspo_new.pkl")  

In [156]:
aspo_full = pd.concat([aspo, new_df_aspo], axis = 1)

In [157]:
aspo_full.to_pickle("../Data/data/aspo_full.pkl") 

In [158]:
aspo_full = pd.read_pickle('../Data/data/aspo_full.pkl')

In [159]:
aspo_new = pd.read_pickle('../Data/data/aspo_new.pkl')

In [160]:
aspo_new[['wildcat_diff','year']][~aspo_new['wildcat_diff'].isna()].query('wildcat_diff > 0')

,wildcat_diff,year
28,99999.000000,1957.0
31,99999.000000,1960.0
34,299999.000000,1963.0
41,199999.000000,1970.0
48,99999.000000,1977.0
...,...,...
5092,0.285714,1981.0
5093,0.222222,1982.0
5096,0.888889,1985.0
5099,0.333333,1988.0


In [161]:
'discoveryaspoPC' in aspo_full.columns

True

In [162]:
aspo_full['wildcat_diff'].isna().sum()

555

In [163]:
aspo_full[['wildcat_diff','year']][~aspo_full['wildcat_diff'].isna()].query('wildcat_diff > 0')

,wildcat_diff,year
28,99999.000000,1957.0
31,99999.000000,1960.0
34,299999.000000,1963.0
41,199999.000000,1970.0
48,99999.000000,1977.0
...,...,...
5092,0.285714,1981.0
5093,0.222222,1982.0
5096,0.888889,1985.0
5099,0.333333,1988.0


In [164]:
aspo_full

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,imputedzero,aspo,wildcat,endowment,...,valoilres_impute_diff,valoilres_impute_binarize,democracy_diff,wildcat_diff,out_regdisaster_diff,oilreserves_diff,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize
0,1929.0,8,0.281900,0.281900,NaN,NaN,NaN,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930.0,8,0.282660,0.282660,NaN,0.00076,NaN,1.0,1.00000,0.8,...,NaN,NaN,NaN,NaN,0.000000e+00,0.002696,NaN,NaN,-0.002409,0.0
2,1931.0,8,0.282660,0.282660,NaN,0.00001,NaN,1.0,0.00001,0.8,...,-0.005175,0.0,0.0,-0.99999,9.525810e+11,0.000000,-0.005175,0.0,-0.006073,0.0
3,1932.0,8,0.282660,0.282660,NaN,0.00001,NaN,1.0,0.00001,0.8,...,0.003964,0.0,0.0,0.00000,-1.000000e+00,0.000000,0.003964,0.0,-0.005035,0.0
4,1933.0,8,0.282649,0.282649,NaN,0.00001,NaN,1.0,0.00001,0.8,...,-0.002133,0.0,0.0,0.00000,3.566128e+10,-0.000039,-0.002133,0.0,-0.005049,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,2004.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.00000,7.267240e-01,0.000000,0.000000,0.0,0.000000,0.0
5116,2005.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.00000,1.625349e+00,0.000000,0.000000,0.0,0.000000,0.0
5117,2006.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.00000,-8.715621e-01,0.000000,0.000000,0.0,0.000000,0.0
5118,2007.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.0,0.00000,1.888971e-01,0.000000,0.000000,0.0,0.000000,0.0


In [166]:
# test: editted by Wanran on Nov 28
# aspo_full[(aspo_full.newdiscovery_aspo>0.00001) & (aspo_full.wildcat==0.00001)]

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,imputedzero,aspo,wildcat,endowment,...,valoilres_impute_diff,valoilres_impute_binarize,democracy_diff,wildcat_diff,out_regdisaster_diff,oilreserves_diff,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize
57,1986.0,8,0.096687,0.096687,NaN,0.00300,NaN,1.0,0.00001,0.80,...,-0.008913,0.0,0.000000,-0.999997,-6.480044e-01,-1.896159e-01,-0.008913,0.0,-0.205402,0.0
92,1941.0,12,0.000660,0.000660,NaN,0.00006,NaN,1.0,0.00001,28.00,...,NaN,NaN,NaN,0.000000,-1.000000e+00,1.000000e-01,NaN,NaN,NaN,NaN
563,1932.0,48,1.185000,1.185000,NaN,1.18500,NaN,1.0,0.00001,1.25,...,NaN,NaN,NaN,0.000000,0.000000e+00,1.184990e+05,NaN,NaN,NaN,NaN
938,1987.0,120,0.956915,0.956915,0.540000,0.00125,NaN,1.0,0.00001,1.50,...,0.001161,0.0,0.000000,-0.999997,3.817416e-01,-5.974162e-02,0.001161,0.0,-0.088896,0.0
942,1991.0,120,0.722767,0.722767,0.400000,0.00500,NaN,1.0,0.00001,1.50,...,-0.003072,0.0,0.000000,0.000000,-4.226914e-01,-6.506246e-02,-0.003072,0.0,-0.091497,0.0
952,2001.0,120,0.377189,0.377189,0.400000,0.02060,NaN,1.0,0.00001,1.50,...,-0.002271,0.0,0.000000,-0.999990,-8.003950e-02,-2.229202e-02,-0.002271,0.0,-0.044939,0.0
1236,1965.0,156,25.983471,25.983471,NaN,0.58500,NaN,1.0,0.00001,60.00,...,-0.000192,0.0,0.000000,-0.999990,-1.888156e-02,2.105927e-02,-0.000192,0.0,-0.002969,0.0
1237,1966.0,156,26.549809,26.549809,NaN,0.64131,NaN,1.0,0.00001,60.00,...,-0.000345,0.0,-0.500000,0.000000,1.601255e-01,2.179611e-02,-0.000345,0.0,-0.006291,0.0
1238,1967.0,156,26.890490,26.890490,NaN,0.40211,NaN,1.0,0.00001,60.00,...,-0.000411,0.0,0.000000,0.000000,-8.026714e-01,1.283181e-02,-0.000411,0.0,-0.012873,0.0
1841,1930.0,276,0.106387,0.106387,NaN,0.00070,NaN,1.0,0.00001,2.50,...,NaN,NaN,NaN,NaN,0.000000e+00,-6.550670e-03,NaN,NaN,-0.011817,0.0
